In [ ]:
# Install dependencies
!pip install textblob
!pip install -U scikit-learn
!pip install nrclex
from google.colab import drive

import sys
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score,confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nrclex import NRCLex
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

drive.mount('/content/drive', force_remount=True)
drive_libs_path = '/content/drive/MyDrive/libs'
sys.path.append(drive_libs_path)

from usage_examples import get_acceptance_indexes, CombinedAcceptanceIndexer
from noise_filter import NoiseFilter

def mount_drive():
    drive.mount('/content/drive', force_remount=True)

def load_and_filter_data(file_path):
    df = pd.read_csv(file_path, low_memory=False,nrows=1000)
    filter_config = {
        'MinLengths': {
            'clean_title': 5,
            'comments': 5
        },
        'ExcludeImages': True
    }
    noise_filter = NoiseFilter(df, filter_config)
    filtered_df = noise_filter.apply()
    filtered_df['clean_title'] = filtered_df['clean_title'].astype(str).str.lower()
    filtered_df['comments'] = filtered_df['comments'].astype(str)
    filtered_df['separated_comment'] = ''
    return filtered_df

def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity, blob.sentiment.subjectivity

def enhanced_preprocess_data(df):
    df['comments_polarity'], df['comments_subjectivity'] = zip(*df['comments'].apply(get_sentiment))
    df['clean_title_polarity'], df['clean_title_subjectivity'] = zip(*df['clean_title'].apply(get_sentiment))
    df['separated_comment'] = df['comments'].apply(lambda x: x.split('|__|'))
    df['acceptance_index'] = df.apply(
        lambda row: calculate_acceptance_index(row['clean_title'], row['comments']),
        axis=1
    )
    return df

def calculate_acceptance_index(title, comments):
    if pd.isna(comments) or comments == '':
        return 0
    comment_list = comments.split('|__|')
    indexer = CombinedAcceptanceIndexer(title, comment_list)
    return indexer.calculate_acceptance_index()

def prepare_features_and_labels(df):
    X = df[['clean_title', 'score', 'separated_comment', 'subreddit',
           'num_comments', 'upvote_ratio', 'comments_polarity',
           'comments_subjectivity', 'clean_title_polarity',
           'clean_title_subjectivity', 'acceptance_index']]

    numerical_features = ['score', 'num_comments', 'upvote_ratio',
                         'comments_polarity', 'comments_subjectivity',
                         'clean_title_polarity', 'clean_title_subjectivity',
                         'acceptance_index']

    for feature in numerical_features:
        if feature not in df.columns:
            df[feature] = 0
        else:
            df[feature] = df[feature].fillna(0)

    X[numerical_features] = X[numerical_features].fillna(0)

    y = df['2_way_label'].astype(str)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numerical_features),
            ('text', TfidfVectorizer(max_features=5000), 'clean_title'),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['subreddit']),
        ])

    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    print("Head of Training DataFrame (X_train):")

    return (X_train, X_test, y_train, y_test), label_encoder, preprocessor

def build_pipeline():
    # Using Linear Regression in the pipeline
    model = Pipeline([
        ('classifier', LinearRegression())
    ])
    return model

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(f"Mean Squared Error (MSE): {mean_squared_error(y_test, y_pred):.2f}")
    print(f"R^2 Score: {r2_score(y_test, y_pred):.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
    print(f"Confusion Matrix: {confusion_matrix(y_test, y_pred)}")

def enhanced_predict_new_comments(model, preprocessor, comments,title):
    title_polarity, title_subjectivity = get_sentiment(title)
    df_comments = pd.DataFrame({
        'clean_title': [title] * len(comments),
        'score': [0] * len(comments),
        'subreddit': ['unknown'] * len(comments),
        'num_comments': [0] * len(comments),
        'upvote_ratio': [0] * len(comments),
        'separated_comment': [''] * len(comments),
        'comments_polarity': [TextBlob(c).sentiment.polarity for c in comments],
        'comments_subjectivity': [TextBlob(c).sentiment.subjectivity for c in comments],
        'clean_title_polarity': [title_polarity] * len(comments),
        'clean_title_subjectivity': [title_subjectivity] * len(comments),
        'acceptance_index': [0] * len(comments)
    })

    comments_transformed = preprocessor.transform(df_comments)
    predictions = model.predict(comments_transformed)

    results = []
    for comment, pred in zip(comments, predictions):
        blob = TextBlob(comment)
        results.append({
            'comment': comment,
            'prediction': pred,
            'polarity': blob.sentiment.polarity,
            'subjectivity': blob.sentiment.subjectivity
        })

    return results

def main():
    mount_drive()
    file_path = "/content/drive/My Drive/datasets/merged_cleaned_data_v31_news.csv"
    df = load_and_filter_data(file_path)
    processed_df = enhanced_preprocess_data(df)
    (X_train, X_test, y_train, y_test), label_encoder, preprocessor = prepare_features_and_labels(processed_df)
    model = build_pipeline()
    model.fit(X_train, y_train)
    evaluate_model(model, X_test, y_test)
    new_title = "Shocking Discovery Leaves Internet Divided—Hoax or Breakthrough?"
    new_comments = [
        "This is outrageous! There's no way this can be true!",
        "I can't believe people are falling for this. It's just a hoax.",
        "Great news! This is exactly what we needed.",
        "Interesting perspective, I hadn't considered that before."
    ]

    predictions = enhanced_predict_new_comments(model, preprocessor, new_comments,new_title)
    print("\nEnhanced Predictions with Sentiment Analysis:")
    for result in predictions:
        print(f"\nComment: {result['comment']}")
        print(f"Prediction: {result['prediction']}")
        print(f"Polarity: {result['polarity']:.2f} (Negative to Positive)")
        print(f"Subjectivity: {result['subjectivity']:.2f} (Objective to Subjective)")

if __name__ == "__main__":
    main()


INFO: pip is looking at multiple versions of nrclex to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nrclex: filename=NRCLex-3.0.0-py3-none-any.whl size=43308 sha256=8bdf81d1fc693861c91eadf91bebc963f9abf9a2b2d650e87f21140d103028f8
  Stored in directory: /root/.cache/pip/wheels/ed/ac/fa/7afddefd14f51c4a963ed291b9052746ed3929473e5a33118d
Successfully built nrclex


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Mounted at /content/drive
Mounted at /content/drive
Head of Training DataFrame (X_train):
Mean Squared Error (MSE): 0.00
R^2 Score: 1.00
RMSE: 0.00
Confusion Matrix: [[175]]

Enhanced Predictions with Sentiment Analysis:

Comment: This is outrageous! There's no way this can be true!
Prediction: 0.0
Polarity: -0.28 (Negative to Positive)
Subjectivity: 0.82 (Objective to Subjective)

Comment: I can't believe people are falling for this. It's just a hoax.
Prediction: 0.0
Polarity: 0.00 (Negative to Positive)
Subjectivity: 0.00 (Objective to Subjective)

Comment: Great news! This is exactly what we needed.
Prediction: 0.0
Polarity: 0.62 (Negative to Positive)
Subjectivity: 0.50 (Objective to Subjective)

Comment: Interesting perspective, I hadn't considered that before.
Prediction: 0.0
Polarity: 0.50 (Negative to Positive)
Subjectivity: 0.50 (Objective to Subjective)


<ipython-input-1-6ff0228387a6>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numerical_features] = X[numerical_features].fillna(0)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
